In [1]:
import os

In [2]:
%pwd

'e:\\Kidney_Disease_Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Kidney_Disease_Classification'

In [5]:
# Entity
# Paste in entity\config_entity.py

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [6]:
from Kidney_Disease_Classification.constrants import * # Import Everything
from Kidney_Disease_Classification.utils.common import read_yaml,create_directories 

In [7]:
# Paste in Configuration (config\configuration.py)

class ConfigurationManager: 
    
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root]) # Prepare Directories

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
          config=self.config.prepare_callbacks
          model_ckpt_dir=os.path.dirname(config.checkpoint_model_filepath)
          create_directories([
                Path(model_ckpt_dir),                     # Check Point Directory
                Path(config.tensorboard_root_log_dir)      # Tensorboard Directory
          ])

          prepare_callback_config=PrepareCallbacksConfig(                           # Convert Path to string
                root_dir=str(config.root_dir),
                tensorboard_root_log_dir=str(config.tensorboard_root_log_dir),
                checkpoint_model_filepath=str(config.checkpoint_model_filepath)
          )
          

          return prepare_callback_config

In [8]:
# Update the components

import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
from pathlib import Path

In [9]:
class PrepareCallbacks:
    def __init__(self,config:PrepareCallbacksConfig):
        self.config=config

    @property
    def _create_tb_callbacks(self):   # Create time stamp folder and create all the logs there
        timestamp=time.strftime("%y-%m-%d-%H-%M-%S")
        tb_running_log_dir=os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):   # Create Check Point call backs
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
    
    def get_tb_ckpt_callbacks(self): # Get the check points along with time
        return[
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]
    
    


In [10]:
# Prepare the Pipeline

try:
    config = ConfigurationManager()
    prepare_callbacks_config=config.get_prepare_callback_config()
    prepare_callbacks=PrepareCallbacks(config=prepare_callbacks_config)
    callback_list=prepare_callbacks.get_tb_ckpt_callbacks()
    

except Exception as e:
    raise e

# Folder created
# Pipeline is not needed for this , we will this in the time of training


[2024-09-03 19:17:10,617: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-03 19:17:10,617: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-03 19:17:10,626: INFO: common: created directory at: artifacts]
[2024-09-03 19:17:10,630: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2024-09-03 19:17:10,631: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
